In [1]:
include("../source/spin-1_XY_ED.jl")

odlro_matrix (generic function with 1 method)

#### 1) Entanglement when $ J_2 = 0 $ 

In [8]:
N = 10
J1,J3,D,h = 1.0,0.1,0.1,1.0
mz = 0
boundary = "OBC"
use_sparse = false

top_folder = joinpath("D:\\IACS_Project\\spin-1_XY\\data\\1D\\spin-1_XY_N=$(N)\\unperturbed\\OBC")
entang_file = joinpath(top_folder,"entanglement",
"entanglement_mz=$(mz)_J1=$(round(J1,digits=6))_J3=$(round(J3,digits=6))_D=$(round(D,digits=6))_h=$(round(h,digits=6)).jld2")
entang_entropy_info = JLD2.load(entang_file)
scar_energy_list = entang_entropy_info["scar_energy"]
scar_entang_entropy_list = entang_entropy_info["scar_entanglement"]
spectrum_energy = entang_entropy_info["spectrum_energy"]
spectrum_entang_entropy_list = entang_entropy_info["spectrum_entanglement"];

In [9]:
using KernelDensity
using StatsPlots

E = spectrum_energy
S_A = spectrum_entang_entropy_list./log(2)

@time begin
    # 2D KDE
    k = kde((E,S_A))
    # evaluate density at each point
    densities = [pdf(k,xx,yy) for (xx,yy) in zip(E,S_A)]
end;

 33.050549 seconds (4.10 M allocations: 14.143 GiB, 4.36% gc time, 8.51% compilation time)


In [41]:
using Plots
using LaTeXStrings
using Plots.PlotMeasures
gr()

plot_size = (700,466.67)
default(fontfamily="Computer Modern")

# plot density-colored scatter
p = scatter(E,S_A,marker_z=densities,c=:plasma,marker=:circle,ms=2.2,msw=0,label=false,colorbar=true,
             size=plot_size,dpi=200)

scatter!(scar_energy_list,(scar_entang_entropy_list./log(2)),marker=:circle,markersize=4,color=:red,markerstrokewidth=0,
         label=false)

random_state_entropy::Float64 = (N/2)*log(3)-0.5
hline!([random_state_entropy./log(2)],color=:black,linestyle=:dash,linewidth=1,label=false)

plot!(left_margin=4mm,right_margin=2mm,top_margin=1mm,bottom_margin=4mm)
plot!(framestyle=:box)
plot!(legend=false,legend_position=(0.1,0.9),legendfontsize=14,legend_background_color=:white,
    legend_foreground_color=:black,legend_spacing=0.5,legend_columns=1)
plot!(grid=false,gridlinewidth=0.6,gridalpha=0.2,gridcolor=:gray)

xlabel!("\$E\$",guidefontsize=21)
ylabel!("\$S_{A}/\\ln2\$",guidefontsize=21)
title_text = "\$ N = $N \\quad $boundary,\\ J_{1} = $(round(J1,digits=6)),\\ J_{3} = $(round(J3,digits=6)), \\ D = $(round(D,digits=6)), \\ h = $(round(h,digits=6)) \$"
# title!(title_text,titlefontsize=16,pad=12)

x1,x_step,x2 = -5.0,5,10.0
y1,y_step,y2 = 1.0,2,7.0
xticks = (x1:x_step:x2,string.(x1:x_step:x2))
yticks = (y1:y_step:y2,string.(y1:y_step:y2))
xminor_ticks = 5 
yminor_ticks = 5
plot!(xticks=xticks,yticks=yticks,xminorgrid=false,yminorgrid=false,
      xminorticks=xminor_ticks, yminorticks=yminor_ticks,tickfontsize=17,tick_direction=:in)
# plot!(xlims=(-0.4,x2+0.4))

folder_name = "D:\\IACS_Project\\spin-1_XY\\plots\\1D\\spin-1_XY_N=$(N)\\unperturbed\\$(boundary)"
if !isdir(folder_name)
    mkpath(folder_name)
end

file_path = joinpath(folder_name,"Entanglement_Entropy_J1=$(J1)_J3=$(round(J3,digits=6))_D=$(round(D,digits=6))_h=$(round(h,digits=6)).png")

# display(p)
# savefig(p,file_path)

LoadError: UndefVarError: `densities` not defined

#### 2) Entanglement Evolution when $ J_2 = 0 $ 

In [2]:
N = 8
J1,J3,D,h = 1.0,0.1,0.1,1.0
boundary = "OBC"
use_sparse = false

time_list::Vector{Float64} = Vector{Float64}((LinRange(0,40,800)))
n_t = length(time_list)
t_step = time_list[2]-time_list[1];

In [3]:
@time begin
    top_folder = joinpath("D:\\IACS_Project\\spin-1_XY\\data\\1D\\spin-1_XY_N=$(N)\\unperturbed\\OBC")
    neel_evolved_file = joinpath(top_folder,"neel_t_evolved_state",
    "evolved_state_J1=$(round(J1,digits=6))_J3=$(round(J3,digits=6))_D=$(round(D,digits=6))_h=$(round(h,digits=6))_tstep=$(round(t_step,digits=6)).jld2")
    ferro_evolved_file = joinpath(top_folder,"ferro_t_evolved_state",
    "evolved_state_J1=$(round(J1,digits=6))_J3=$(round(J3,digits=6))_D=$(round(D,digits=6))_h=$(round(h,digits=6))_tstep=$(round(t_step,digits=6)).jld2")
    prod_evolved_file = joinpath(top_folder,"prod_t_evolved_state",
    "evolved_state_J1=$(round(J1,digits=6))_J3=$(round(J3,digits=6))_D=$(round(D,digits=6))_h=$(round(h,digits=6))_tstep=$(round(t_step,digits=6)).jld2")
    neel_evolved_state_list = JLD2.load(neel_evolved_file)["evolved_state_list"]
    ferro_evolved_state_list = JLD2.load(ferro_evolved_file)["evolved_state_list"]
    prod_evolved_state_list = JLD2.load(prod_evolved_file)["evolved_state_list"]
end;

  4.516445 seconds (1.26 M allocations: 324.723 MiB, 3.27% gc time, 42.53% compilation time: 91% of which was recompilation)


In [4]:
neel_state_entropy_list::Vector{Float64} = Vector{Float64}(undef,n_t)
ferro_state_entropy_list::Vector{Float64} = Vector{Float64}(undef,n_t)
prod_state_entropy_list::Vector{Float64} = Vector{Float64}(undef,n_t)
N_A = N_B = Int64(N/2)

@time begin
    for i in 1:n_t
        neel_t_evolved_state = neel_evolved_state_list[i]
        ferro_t_evolved_state = ferro_evolved_state_list[i]
        prod_t_evolved_state = prod_evolved_state_list[i]
        neel_state_entropy_list[i] = bipartite_entang_entropy_full(neel_t_evolved_state,1,N_A,N_B)
        ferro_state_entropy_list[i] = bipartite_entang_entropy_full(ferro_t_evolved_state,1,N_A,N_B)
        prod_state_entropy_list[i] = bipartite_entang_entropy_full(prod_t_evolved_state,1,N_A,N_B)
    end
end;

  6.468169 seconds (315.46 k allocations: 484.974 MiB, 2.82% gc time, 2.73% compilation time)


In [39]:
using Plots
using LaTeXStrings
using Plots.PlotMeasures
gr()

plot_size = (700,466.67)
default(fontfamily="Computer Modern")

p = scatter(h*time_list,(neel_state_entropy_list./log(2)),marker=:circle,markercolor=:white,markerstrokecolor=:navy,markerstrokewidth=1,
        ms=3,lw=0.7,label="Nematic Néel state",size=plot_size,dpi=200)

scatter!(h*time_list,(ferro_state_entropy_list./log(2)),marker=:circle,markercolor=:white,markerstrokecolor=:green,markerstrokewidth=1,
        ms=3,lw=0.7,label="Nematic ferro state")

scatter!(h*time_list,(prod_state_entropy_list./log(2)),marker=:circle,markercolor=:white,markerstrokecolor=:red,markerstrokewidth=1,
        ms=3,lw=0.7,label="\$S_{z}\$ product state")

random_state_entropy::Float64 = (N/2)*log(3)-0.5
hline!([random_state_entropy./log(2)],color=:black,linestyle=:dash,linewidth=1,label=false)

plot!(left_margin=4mm,right_margin=1mm,top_margin=1mm,bottom_margin=2mm)
plot!(framestyle=:box)
plot!(grid=false,gridlinewidth=0.6,gridalpha=0.2,gridcolor=:gray)
plot!(legend_position=(0.64,0.25),legendfontsize=14,legend_background_color=:white,
    legend_foreground_color=:black,legend_frame=true,legend_spacing=0.5,legend_columns=1)

xlabel!("\$ht\$",guidefontsize=21)
ylabel!("\$S_{A}(t)/\\ln2\$",guidefontsize=21)
title_text = "\$ N = $N \\quad $boundary,\\ J_{1} = $(round(J1,digits=6)),\\ J_{3} = $(round(J3,digits=6)), \\ D = $(round(D,digits=6)), \\ h = $(round(h,digits=6)) \$"
# title!(title_text,titlefontsize=16)

x1,x_step,x2 = 0,10,40
y1,y_step,y2 = 0.0,1,5.0
xticks = (x1:x_step:x2,string.(x1:x_step:x2))
yticks = (y1:y_step:y2,string.(y1:y_step:y2))
xminor_ticks = 2 
yminor_ticks = 2
plot!(xticks=xticks,yticks=yticks,xminorgrid=false,yminorgrid=false,
      xminorticks=xminor_ticks, yminorticks=yminor_ticks,tickfontsize=17,tick_direction=:in)
plot!(xlims=(-0.4,x2+0.4))

folder_name = "D:\\IACS_Project\\spin-1_XY\\plots\\1D\\spin-1_XY_N=$(N)\\unperturbed\\$(boundary)"
if !isdir(folder_name)
    mkpath(folder_name)
end

file_path = joinpath(folder_name,"Entanglement_Evolution_J1=$(round(J1,digits=6))_J3=$(round(J3,digits=6))_D=$(round(D,digits=6))_h=$(round(h,digits=6)).png")

# display(p)
# savefig(p,file_path)

"D:\\IACS_Project\\spin-1_XY\\plots\\1D\\spin-1_XY_N=8\\unperturbed\\OBC\\Entanglement_Evolution_J1=1.0_J3=0.1_D=0.1_h=1.0.png"

#### 3) Entanglement Evolution when $ J_2 \neq 0 $ 

In [5]:
N = 8
J1,J3,D,h = 1.0,0.1,0.1,1.0
flag = "full"
boundary = "OBC"
use_sparse = false

time_list::Vector{Float64} = Vector{Float64}((LinRange(0,400,4000)))
n_t = length(time_list)
t_step = time_list[2]-time_list[1];

In [6]:
size_A = size_B = Int64(N/2)

@time begin
    J2_list::Vector{Float64} = Vector{Float64}([0.02,0.03,0.05,0.07,0.2])
    J2_neel_entropy_list::Vector{Vector{Float64}} = Vector{Vector{Float64}}(undef,length(J2_list))
    
    for (pos,J2) in enumerate(J2_list)    
        top_folder = joinpath("D:\\IACS_Project\\Spin-1_XY\\Data\\1D\\spin-1_XY_N=$(N)\\sec_nbr_perturbed\\OBC")
        neel_evolved_file = joinpath(top_folder,"neel_t_evolved_state",
        "evolved_state_J1=$(round(J1,digits=6))_J3=$(round(J3,digits=6))_J2=$(round(J2,digits=6))_D=$(round(D,digits=6))_h=$(round(h,digits=6))_tstep=$(round(t_step,digits=6)).jld2")
        neel_evolved_state_list = JLD2.load(neel_evolved_file)["evolved_state_list"]        

        neel_state_entropy_list::Vector{Float64} = Vector{Float64}(undef,n_t)
        for i in 1:n_t
            neel_t_evolved_state = neel_evolved_state_list[i]
            neel_state_entropy_list[i] = bipartite_entang_entropy_full(neel_t_evolved_state,1,size_A,size_B)
        end
        J2_neel_entropy_list[pos] = neel_state_entropy_list
    end
end;

 71.538859 seconds (938.97 k allocations: 5.777 GiB, 3.99% gc time, 0.06% compilation time)


In [14]:
using Plots
using LaTeXStrings
using Plots.PlotMeasures
gr()

plot_size = (700,466.67)
default(fontfamily="Computer Modern")

J2_1 = round(J2_list[1],digits=3)
J2_2 = round(J2_list[2],digits=3)
J2_3 = round(J2_list[3],digits=3)
J2_4 = round(J2_list[4],digits=3)
J2_5 = round(J2_list[5],digits=3)
# J2_6 = round(J2_list[6],digits=3)

p = scatter(h*time_list,(J2_neel_entropy_list[1]./log(2)),marker=:circle,markercolor=:white,markerstrokecolor=:navy,markerstrokewidth=1,
        ms=2.5,label="\$\\lambda=$(J2_1)\$",size=plot_size,dpi=200)

scatter!(h*time_list,(J2_neel_entropy_list[2]./log(2)),marker=:circle,markercolor=:white,markerstrokecolor=:green,markerstrokewidth=1,
        ms=2.5,label="\$\\lambda=$(J2_2)\$")

scatter!(h*time_list,(J2_neel_entropy_list[3]./log(2)),marker=:circle,markercolor=:white,markerstrokecolor=:red,markerstrokewidth=1,
        ms=2.5,label="\$\\lambda=$(J2_3)\$")

scatter!(h*time_list,(J2_neel_entropy_list[4]./log(2)),marker=:circle,markercolor=:white,markerstrokecolor=:purple,markerstrokewidth=1,
        ms=2.5,label="\$\\lambda=$(J2_4)\$")

scatter!(h*time_list,(J2_neel_entropy_list[5]./log(2)),marker=:circle,markercolor=:white,markerstrokecolor=:orange,markerstrokewidth=1,
        ms=2.5,label="\$\\lambda=$(J2_5)\$")

# scatter!(h*time_list,(J2_neel_entropy_list[6]./log(2)),marker=:circle,markercolor=:white,markerstrokecolor=:cyan,markerstrokewidth=1,
#         ms=2.5,label="\$\\lambda=$(J2_6)\$")

random_state_entropy::Float64 = (N/2)*log(3)-0.5
hline!([random_state_entropy./log(2)],color=:black,linestyle=:dash,linewidth=1,label=false)

plot!(left_margin=5mm,right_margin=4mm,top_margin=1mm,bottom_margin=4mm)
plot!(framestyle=:box)
plot!(grid=false,gridlinewidth=0.6,gridalpha=0.2,gridcolor=:gray)
plot!(legend_position=(0.82,0.36),legendfontsize=15,legend_background_color=:white,
    legend_foreground_color=:black,legend_frame=true,legend_spacing=0.5,legend_columns=1)

xlabel!("\$ht\$",guidefontsize=21)
ylabel!("\$S_{A}(t)/\\ln2\$",guidefontsize=21)
title_text = "\$ N = $N \\quad $boundary,\\J_{1} = $(round(J1,digits=6)),\\ J_{3} = $(round(J3,digits=6)), \\ D = $(round(D,digits=6)), \\ h = $(round(h,digits=6)) \$"
# title!(title_text,titlefontsize=16)

x1,x_step,x2 = 0,50,300
y1,y_step,y2 = 0.0,1.0,5.0
xticks = (x1:x_step:x2,string.(x1:x_step:x2))
yticks = (y1:y_step:y2,string.(y1:y_step:y2))
xminor_ticks = 2
yminor_ticks = 2
plot!(xticks=xticks,yticks=yticks,xminorgrid=false,yminorgrid=false,
      xminorticks=xminor_ticks,yminorticks=yminor_ticks,tickfontsize=17,tick_direction=:in)
plot!(xlims=(-2.5,x2+2))

folder_name = "D:\\IACS_Project\\spin-1_XY\\plots\\1D\\spin-1_XY_N=$(N)\\sec_nbr_perturbed\\$(boundary)"
if !isdir(folder_name)
    mkpath(folder_name)
end

file_path = joinpath(folder_name,"Entanglement_Evolution_J1=$(round(J1,digits=6))_J3=$(round(J3,digits=6))_J2_D=$(round(D,digits=6))_h=$(round(h,digits=6)).png")

# display(p)
# savefig(p,file_path)

"D:\\IACS_Project\\spin-1_XY\\plots\\1D\\spin-1_XY_N=8\\sec_nbr_perturbed\\OBC\\Entanglement_Evolution_J1=1.0_J3=0.1_J2_D=0.1_h=1.0.png"